In [23]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


In [24]:
ds = load_dataset("dwisaji/indonesia-telecomunication-sentiment-dataset")
train_data, val_data = train_test_split(ds['train'], test_size=0.2, random_state=42, stratify=ds['train']['label'])

print(f"Training data: {len(train_data)} columns")
print(f"Validation data: {len(val_data)} columns")

Training data: 2 columns
Validation data: 2 columns


In [ ]:
# from datasets import load_dataset, ClassLabel

# # Load dataset
# ds = load_dataset("dwisaji/indonesia-telecomunication-sentiment-dataset")

# # Konversi kolom 'label' ke ClassLabel
# label_names = ['Negatif', 'Netral', 'Positif']
# ds = ds.cast_column('label', ClassLabel(names=label_names))

# # Split dataset dengan stratifikasi
# train_val_split = ds['train'].train_test_split(
#     test_size=0.2,
#     seed=42,
#     stratify_by_column='label'
# )
# train_data = train_val_split['train']
# val_data = train_val_split['test']

# # Lanjutkan dengan tokenisasi dan training seperti sebelumnya

In [ ]:
# ds = load_dataset("dwisaji/indonesia-telecomunication-sentiment-dataset")


# train_data, val_data = train_test_split(ds['train'], test_size=0.2, random_state=42, stratify=ds['train']['label'])

# print(f"Training data: {len(train_data)} columns")
# print(f"Validation data: {len(val_data)} columns")

Training data: 2 columns
Validation data: 2 columns


In [5]:
# def tokenize()

In [15]:
model_name = "indolem/indobert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name, truncation=True)


model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3,
    id2label=id_to_labels,
    label2id=label_to_id
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indolem/indobert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
def preprocess_func(sample):
    return tokenizer(sample['text'], truncation=True, max_length=512)

In [12]:
tokenized_train = train_data.map(preprocess_func, batched=True)
tokenized_val = val_data.map(preprocess_func, batched=True)

Map: 100%|██████████| 495/495 [00:00<00:00, 18706.18 examples/s]


In [20]:
data_collocator = DataCollatorWithPadding(tokenizer=tokenizer)

In [13]:
tokenized_train = tokenized_train.remove_columns(['text']).set_format("torch")
tokenized_val = tokenized_val.remove_columns(['text']).set_format("torch")

In [17]:
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)
    acc = accuracy.compute(predictions=preds, references=labels)['accuracy']
    f1_score = f1.compute(predictions=preds, references=labels, average='weighted')['f1']
    return {"accuracy": acc, "f1": f1_score}

In [18]:
training_args = TrainingArguments(
    output_dir="indobert-sentimen",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)

c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collocator,
    compute_metrics=compute_metrics
)

C:\Users\alber\AppData\Local\Temp\ipykernel_14508\257162527.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


ValueError: You have set `args.eval_strategy` to IntervalStrategy.EPOCH but you didn't pass an `eval_dataset` to `Trainer`. Either set `args.eval_strategy` to `no` or pass an `eval_dataset`. 

In [21]:
trainer.train()

KeyError: 0